In [10]:
import json
from pprint import pprint
from analysis.gender import gender, gender_special
from analysis.utils import get_people_mentioned, get_gender, get_sources

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def process_coref_chain(coref_id, coref_chain):
    # Get the unique representative element
    rep_elem = next(coref for coref in coref_chain if coref['isRepresentativeMention'])
    to_print = [(elem['text'], elem['type'], elem['number'], elem['gender'], elem['sentNum']) for elem in coref_chain
                if elem['animacy'] == 'ANIMATE']
    if len(to_print):
        print len(coref_chain), to_print

In [4]:
def load_tc_data(year, month, folder='annotated/'):
    tc_data = {}
    with open('{}techcrunch_annotated_{}_{}.tsv'.format(folder, year, month), 'r') as tc_f:
        for line in tc_f:
            link, data, corenlp = line.strip().split('\t')
            tc_data[link] = {'data': json.loads(data), 'corenlp': json.loads(corenlp)}
    
    return tc_data

In [5]:
def get_mentions_counts(data):
    all_counts = {'MALE': {'MALE': 0, 'FEMALE': 0}, 'FEMALE': {'MALE': 0, 'FEMALE': 0}, 'UNK': {'MALE': 0, 'FEMALE': 0}}
    for link, data in data.iteritems():
        try:
            gend = get_gender(data['author'])
            if type(gend) is str:
                gend = gend.upper()
            else:
                gend = 'UNK'
            if not gend:
                continue
            counts = all_counts[gend]
            sentences = data['corenlp']['sentences']
            corefs = data['corenlp']['corefs']
            people_mentioned = process_sentences(sentences)
            pm_to_gender = {pm: None for pm in people_mentioned}
            for c_id, coref_chain in corefs.iteritems():
                rep_elem = next(coref for coref in coref_chain if coref['isRepresentativeMention'])
                elem_text = set(rep_elem['text'].split())
                for pm in pm_to_gender:
                    if len(set.intersection(elem_text, set(pm))) > 0:
                        if rep_elem['gender'] in ['MALE', 'FEMALE']:
                            pm_to_gender[pm] = rep_elem['gender']
            for gender in ['MALE', 'FEMALE']:
                counts[gender] += len([sp for sp in pm_to_gender.values() if sp == gender])
        except:
            continue
    return all_counts

In [8]:
for year in range(2009, 2017):
    with open('data/annotated_old/techcrunch_annotated_{}.json'.format(year), 'r') as tc_f:
        data = json.load(tc_f)
        print year, get_mentions_counts(data)

2009 {'UNK': {'MALE': 4006, 'FEMALE': 440}, 'MALE': {'MALE': 9721, 'FEMALE': 1066}, 'FEMALE': {'MALE': 2520, 'FEMALE': 301}}
2010 {'UNK': {'MALE': 5241, 'FEMALE': 567}, 'MALE': {'MALE': 10354, 'FEMALE': 1047}, 'FEMALE': {'MALE': 4902, 'FEMALE': 537}}
2011 {'UNK': {'MALE': 5147, 'FEMALE': 544}, 'MALE': {'MALE': 9420, 'FEMALE': 970}, 'FEMALE': {'MALE': 8884, 'FEMALE': 1025}}
2012 {'UNK': {'MALE': 4406, 'FEMALE': 441}, 'MALE': {'MALE': 10956, 'FEMALE': 1223}, 'FEMALE': {'MALE': 7407, 'FEMALE': 925}}
2013 {'UNK': {'MALE': 3397, 'FEMALE': 346}, 'MALE': {'MALE': 14846, 'FEMALE': 1615}, 'FEMALE': {'MALE': 9682, 'FEMALE': 1232}}
2014 {'UNK': {'MALE': 954, 'FEMALE': 121}, 'MALE': {'MALE': 14318, 'FEMALE': 1582}, 'FEMALE': {'MALE': 7363, 'FEMALE': 1210}}
2015 {'UNK': {'MALE': 614, 'FEMALE': 72}, 'MALE': {'MALE': 13075, 'FEMALE': 1553}, 'FEMALE': {'MALE': 7555, 'FEMALE': 1399}}
2016 {'UNK': {'MALE': 274, 'FEMALE': 44}, 'MALE': {'MALE': 7386, 'FEMALE': 881}, 'FEMALE': {'MALE': 4755, 'FEMALE': 950}

In [13]:
MIN_NUM = 623 + 6
MAX_NUM = MIN_NUM + 1
curr_idx = 0
for link, data in tc_2016_data.iteritems():
    print data['author'], get_gender(data['author'])
    continue
    curr_idx += 1
    if curr_idx < MIN_NUM:
        continue
    if curr_idx >= MAX_NUM:
        break    
    print link
    corefs = data['corenlp']['corefs']
    print corefs['43']
    sentences = data['corenlp']['sentences']
    # print sentences[0].keys()
    # pprint(sentences[5]['tokens'])
    print process_sentences(sentences)
    for c_id, coref in corefs.iteritems():
        process_coref_chain(c_id, coref)

NameError: name 'tc_2016_data' is not defined

In [6]:
tc_data = load_tc_data(2016, 1)

In [19]:
MIN_NUM = 11
MAX_NUM = MIN_NUM + 10
curr_idx = 0
for link, values in tc_data.iteritems():
    curr_idx += 1
    if curr_idx < MIN_NUM:
        continue
    if curr_idx > MAX_NUM:
        break

    # if link != 'https://techcrunch.com/2016/01/05/intel-says-button-sized-curie-will-ship-in-q1-costing-under-10/':
    #    continue
    data = values['data']
    corenlp = values['corenlp']
    print link
    corefs = corenlp['corefs']
    sentences = corenlp['sentences']
    # pprint(corefs)
    # pprint(sentences[5])
    # print sentences[0].keys()
    # pprint(sentences[5]['tokens'])
    pm = get_people_mentioned(sentences, corefs, include_gender=True)
    sources = get_sources(pm, sentences, corefs)
    print pm
    print {k: len(v) for k, v in sources.iteritems()}
    #for c_id, coref in corefs.iteritems():
    #    process_coref_chain(c_id, coref)

https://techcrunch.com/2016/01/14/nasa-receives-patent-for-a-new-type-of-squishy-amorphous-robot/
{u'Imagecourtesy': (5, (None, None)), u'Arthur Bradley': (1, (u'MALE', 'COREF'))}
{u'Imagecourtesy': 0, u'Arthur Bradley': 1}
https://techcrunch.com/2016/01/20/apple-releases-music-memos-a-recorder-app-for-musicians/
{}
{}
https://techcrunch.com/2016/01/16/notify-nearby-launch/
{u'Nevin Jethmalani': (3, (u'MALE', 'COREF')), u'Levi': (1, ('male', 'NAME_ONLY'))}
{u'Nevin Jethmalani': 84, u'Levi': 0}
https://techcrunch.com/2016/01/27/watch-microsoft-ventures-london-accelerator-right-here-2/
{}
{}
https://techcrunch.com/2016/01/18/why-big-companies-keep-failing-the-stack-fallacy/
{u'Anshu Sharma': (1, ('male', 'NAME_ONLY')), u'Larry Ellison': (1, (u'MALE', 'COREF'))}
{u'Anshu Sharma': 0, u'Larry Ellison': 0}
https://techcrunch.com/2016/01/26/ipo-slowdown-a-look-at-company-profitability/
{u'Ben Narasin': (1, ('male', 'NAME_ONLY')), u'Jeremy': (1, ('male', 'NAME_ONLY'))}
{u'Ben Narasin': 0, u'Je